<h3>Import Required Packages & Data</h3>

In [1]:
#import required packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [20]:
#import file
file = 'Chicago_Crimes_2016.csv'
df_crime = pd.read_csv(file)

FileNotFoundError: [Errno 2] No such file or directory: 'Chicago_Crimes_2016.csv'

<h3>Review the Data</h3>

In [ ]:
#review a summary of the dataframe
df_crime.info()

<b>Observation #1:</b> There is an unnamed column. Depending on what values are in this column, we may be able to drop it from our dataset.</br></br>
<b>Observation #2:</b> Several columns have missing data: Case Number, Location Description, District, Ward, Community Area, X Coordinate, Y Coordinate, Latitude, Longitude, & Location. We will need to detemine whether or not we want to drop these rows/columns or replace the NULL values with the mean value.</br></br>
<b>Observation #3:</b> There are several columns that seem to revolve around the location of the crime (Location Description, District, Community Area, X Coordinate, Y Coordinate, Latitude, Longitude, & Location). We may be able to drop a few of these columns from our dataset.</br></br>
<b>Observation #4:</b> The <i>Date</i> and <i>Updated On</i> columns are currently labeled as "object." We should update the data types appropriately. Additionally, the <i>Year</i> column may be irrelevant, as the year is already within the <i>Date</i> field.

In [ ]:
#review a sample of the data within the dataframe
df_crime.head(10)

<b>Observation #5:</b> The <i>Date</i> and <i>Updated On</i> columns contain both date and time. We need to consider this when updating the data type.</br></br>
<b>Observation #6:</b> The <i>Location</i> column contains a concatination of <i>Latitude</i> and <i>Longitude</i>. We can consider dropping either just the <i>Location</i> column, or both the <i>Latitude</i> and <i>Longitude</i> columns.</br></br>
<b>Observation #7:</b> The <i>District, Ward, Community, X Coordinate,</i> and <i>Y Coordinate</i> columns are set as "float," however "integer" may be a more appropriate data type.</br></br>
<b>Observation #8:</b> The <i>Block</i> field contains street names. We may be able to create a new column for streets instead of block, which should help to standardize the data.</br></br>
<b>Observation #9:</b> According to Google, <i>IUCR</i> stands for "Illinois Uniform Crime Reporting." This column may not be relevant to our objective and may be able to be dropped.

In [ ]:
#review a summary of the data within the dataframe
df_crime.describe()

<b>Observation #10:</b> There's not much that the describe function tells us at this time.

In [ ]:
pd.set_option('display.max_rows',None)
df_crime["Location Description"].value_counts()

<b>Observation #11:</b> There are 142 different values within the Location Description column. We should consider binning these for our model.

<h3>Wrangle the Data</h3>

In [ ]:
#remove unnecessary columns
df_crime.drop(["Unnamed: 0","ID","Case Number","Location", "Year", "IUCR", "X Coordinate", "Y Coordinate"], axis=1, inplace=True)
    
#update datatypes
#df_crime['Date'] = pd.to_datetime(df_crime['Date'])

#review the dataframe info to ensure accuracy of wrangling
df_crime.info()